<a href="https://colab.research.google.com/github/savindumahasen/AI-Detection-System-/blob/main/Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# install the necessary libraries
!pip install pandas
!pip install numpy
!pip install opendatasets

In [23]:
# import necessary libraries
import pandas as pd
import numpy as np
import opendatasets as od
import csv

In [17]:
## download the dataset
od.download("https://www.kaggle.com/datasets/shanegerami/ai-vs-human-text")

Skipping, found downloaded files in "./ai-vs-human-text" (use force=True to force download)


In [30]:
## load the datast and read first five records
dataset=pd.read_csv("/content/AI_Human.csv", engine="python", on_bad_lines="skip")
dataset.head()

,text,generated
0,Cars. Cars have been around since they became ...,0.0
1,Transportation is a large necessity in most co...,0.0
2,"""America's love affair with it's vehicles seem...",0.0
3,How often do you ride in a car? Do you drive a...,0.0
4,Cars are a wonderful thing. They are perhaps o...,0.0
